# 딕셔너리와 집합

dict형은 파이썬 구현의 핵심
해시 테이블 >> 고성능 딕셔너리, 집합 구현

## 3.1. 일반적인 매핑형

collections.abc 모듈 ; dict,dict 유사 자료형의 인터페이스 정의 위해 mapping, mutablemapping ABC(추상 베이스 클래스) 제공

mutablemapping >> mapping >> container/ iterable / sized (서브클래스 >> 슈퍼클래스)
*특화된 매핑은 위의 ABC 대신 dict나 collections.UserDict 클래스 상속하기도* 
ABC는 (매핑이 제공해야 하는) 최소한의 인터페이스를 정의하고 문서화하기 위한 것. isinstance() 테스트를 하기 위한 기준으로도 사용. 

표준 라이브러리 제공 매핑형은 모두 dict 이용해 구현 >> 키 해시 가능해야
*변하지 않는 해시값, 다른 객체와 비교 가능 : 해시 가능한 객체
원자적 불변형 시뭔스(str, byte, 수치형) : 모두 해시 가능
frozenset : 언제나 해시 가능
튜플 : 들어 있는 모든 항목이 해시 가능해야 해시 가능
사용자 정의 자료형 : 해시 가능 ; id()를 이용해 객체의 해시값 구하기 때문*

## 3.2. 지능형 딕셔너리

지능형 딕셔너리는 모든 반복형 객체에서 키-값 쌍을 생성>> 딕셔너리 객체 만들 수 있음

## 3.3. 공통적인 매핑 메서드

*책의 표(dict, defaultdict, OrderedDict 클래스가 구현하는 메서드) 참고*
update() m 다루는 방식 ; 덕 타이핑

### 3.3.1. setdefault()


In [ ]:
#존재하지 않는 키 처리

#
if key not in my_dict :
    my_dict[key]=[]
my_dict[key].append(new_value)

#setdefault() 이용
my_dict.setdefault(key,[]).append(new_value)  #한번만 검색해서 속도 빠름


## 3.4 키를 조회하는 매핑

검색 > 키 존재x > 어떤 특별한 값 반환하는 매핑 만드는 법
    1.dict 대신 defaultdict를 사용하거나
    2.dict 등의 매핑형 상속 >> __missing__() 메서드 추가
    
### 3.4.1. defaultdict

defaultdict는 존재하지 않는 키로 검색> 항목 생성 하도록 설정
defaultdict 생성 >> 존재하지 않느 키 인수 >> __getitem__() 호출>>기본값 생성 위한 콜러블 제공
*기본값 생성하는 콜러블은 객체 속성 default_factory에 저장
default_factory 설정x >> KeyError 발생*

In [ ]:
dd=defaultdict(list)
dd['new-key'] #'new-key'는 존재하지 않는 키
#리스트 새로 생성 위해 list() 호출
#'new-key' 키로 사용 >> 새로운 리스트 dd에 삽입
#리스트에 대한 참조 반환

### 3.4.2. __missing__() 메서드

매핑형은 __missing__()를 이용해 존재하지 않는 키 처리
기본 클래스 dict에는 정의x
dict 클래스 상속 >> __missing__ 정의 >>> dict.__getitem__()이 키 발견x>>__missing__() 호출

검색>> 키>str로 변환하는 매핑

In [ ]:
# 조회할 때 키를 문자열로 변환하는 StrKeyDict0

class StrKeyDict0(dict):
    
    def __missing(self, key):
        if isinstance(key, str):  #str(k) 존재하지 않으면 무한히 재귀적으로 호출되는 거 방지
            raise KeyError(ky)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

## 3.5 그 외 매핑형

collections.OrderedDict
키 삽입 순서대로 유지. 항목 반복 순서 예측 가능. popitem()은 최근 삽입 항목 꺼냄. my_odict.popitem(last=True)는 처음 삽입한 항목 꺼냄.

collections.ChainMap
매핑들의 목록을 담고 있음. 한꺼번에 모두 검색 가능. 차례대로 검색>>하나라도 키 검색됨>>성공. 인터프리터 구현시 사용가능.

collections.Counter
모든 키에 정수형 카운터를 가지고 있음. 기존 키 갱신>>카운터 늘어남. 카운터는 객체 수 셀 때 사용 가능. 

collections.UserDict
표준 dict처럼 작동하는 매핑을 순수 파이썬으로 구현함.

## 3.6. UserDict 상속

dict보다 UserDict 상속해 매핑형 만들기가 더 쉬움
내장형 ; 메서드들을 오버라이드해야하는 구현 특이성 때문에 UserDict 상속이 낫다.

UserDict는 dict 상속 x. data라고 하는 dict 객체 갖고 있음.
>> 특수 메서드 구현할 때 불필요한 재귀적 호출 피할 수 O

UserDict는 MutableMapping 상속함.>> 매핑의 모등 기능을 가지게 됨.
특히 MutableMapping.update()  와 Mapping.get() 유용함.
*MutableMapping.update() : 직접호출/__init__()에 의해 호출. __setitem__() 호출함.*


In [ ]:
# 삽입, 갱신, 조회할 때 비문자열키를 문자열로 변환하는 StrKeyDict

import collections

class StrKeyDict(collections, UserDict): #UserDict 상속>>MutableMapping or Mapping 상속
    
    def __missing(self, key):
        if isinstance(key, str): 
            raise KeyError(ky)
        return self[str(key)]
        
    def __contains__(self, key):
        return str(key) in self.data  #키가 모두 str이라 self.data에서 바로 조회 가능
    
    def __setitem__(self, key, item): #모든 키를 str로 반환
        self.data[str(key)] = item

## 3.7. 불변 매핑

board.pins : 물리적 범용입출력 핀을 나타냄. 하드웨어를 소프트웨어로 변경 못함 >> 불변 매핑

types 모듈 : MappingProxyType 래퍼 클래스 제공>>읽기 전용의 mappingproxy 객체 반환.
원래 매핑 변경 >> mappingproxy에 반영 but, mappingproxy 직접 변경 불가능


In [ ]:
#MappingProxyType

from types import MappingProxyType
d={1: 'A'}
d_proxy=MappingProxyType(d)
d_proxy
#mappingproxy({1:'A'})
d_proxy[1]
#'A'
d_proxy[2]='x' #직접 변경 x
#Traceback (most recent call last) :
#    File "<stdin>", line 1, in <module>
#TypeError : 'mappingproxy' object odes not support item assignment
d[2]='B'
d_proxy
#mappingproxy({1:'A',2:'B'})
d_proxy[2]
#'B'

## 3.8. 집합 이론

집합 : 고유한 객체의 모음, 중복 항목 제거 , 집합 요소들은 해시할 수 있어야

### 3.8.1 집합 리터럴
공집합 : set()으로 표현해야
공집합 외의 집합 : {}나 set([])이용

### 3.8.2 지능형 집합

예시. [chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

### 3.8.3. 집합 연산

책의 그림, 표 참고

## 3.9. dict 와 set의 내부 구조

### 3.9.1. 성능
dict와 set 시간 매우 적게 걸림.

### 3.9.2. 딕셔너리 안의 해시 테이블

해시 테이블 : 중간에 빈 항목을 가진 배열
dict 해시 테이블에는 각 항목 별로 버킷 있음.
*버킷 : 해시 테이블 안의 항목, 키에 대한 참조와 항목의 값에 대한 참조가 들어감. 모든 버킷의 크기 동일
파이썬은 버킷 1/3 이상 비워두려고 함*

hash() 내장함수 : 내장 자료형 직접 처리, 사용자 정의 자료형>>__hash__()호출

해시 테이블 알고리즘 : 키에서 해시 계산>>해시 일부 이용해 버킷의 위치 찾기>> 빈 버킷 >> KeyError
키에서 해시 계산>>해시 일부 이용해 버킷의 위치 찾기>> 빈 버킷X >> 키 동일X >> 해시 충돌 >> 해시 다른 부분 이용>> 해시 테이블의 다른 위치>>빈 버킷?
*해시충돌 : search_key, found_key 다름*
키에서 해시 계산>>해시 일부 이용해 버킷의 위치 찾기>> 빈 버킷X >> 키 동일>>버킷 안의 값 반환

### 3.9.3. dict 작동 방식에 의한 영향

해시 가능한 객체의 요건
1. 객체의 수명 주기 동안 언제나 동일한 값 반환하는 __hash__() 제공>> hash() 함수 지원
2. __eq__() 통해 동치성 판단 가능
3. a==b >> hash(a)==hash(b)

dict의 메모리 오버헤드 크다. : 공간 효율성 높지 않음..

키 검색 아주 빠름. *속도 위해 공간 포기*

키 순서는 삽입 순서에 다라 달라짐.

딕셔너리 항목 추가 >> 기존 키 순서 바뀔 수 있음

### 3.9.4. 집합의 작동 방식

3.9.3.의 dict 작동 결정 방식이 집합에도 모두 적용

set 요소는 모두 해시 가능한 객체여야
set의 메모리 오버헤드 상당히 큼
집합에 속해있는지 효율적으로 검사 가능
요소 순서는 추가 순서에 따라 달라짐
요소를 집합에 추가 >> 다른 요소의 순서 바뀔 수 있음.